# Fine-tune a BLOOM-based ad generation model using `peft`, `transformers` and `bitsandbytes`

We can use the [Product Descriptions and Ads Dataset](https://huggingface.co/datasets/c-s-ale/Product-Descriptions-and-Ads) to fine-tune BLOOM to be able to generate simple ads based off of product names, and descriptions! Perfect for Twitter or Instagram!

### Overview of PEFT and LoRA:

Based on some awesome new research [here](https://github.com/huggingface/peft), we can leverage techniques like PEFT and LoRA to train/fine-tune large models a lot more efficiently.

It can't be explained much better than the overview given in the above link:

```
Parameter-Efficient Fine-Tuning (PEFT) methods enable efficient adaptation of
pre-trained language models (PLMs) to various downstream applications without
fine-tuning all the model's parameters. Fine-tuning large-scale PLMs is often
prohibitively costly. In this regard, PEFT methods only fine-tune a small
number of (extra) model parameters, thereby greatly decreasing the
computational and storage costs. Recent State-of-the-Art PEFT techniques
achieve performance comparable to that of full fine-tuning.
```

### Install requirements



In [1]:
!pip install -q bitsandbytes datasets accelerate loralib
!pip install -q git+https://github.com/huggingface/transformers.git@main git+https://github.com/huggingface/peft.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 42.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 27.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


### Model loading

Here let's load the `bloom-1b7` model!

In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"
import torch
import torch.nn as nn
import bitsandbytes as bnb
from transformers import AutoTokenizer, AutoConfig, AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained(
    "bigscience/bloom-1b7",
    load_in_8bit=True,
    device_map='auto',
)

tokenizer = AutoTokenizer.from_pretrained("bigscience/tokenizer")

config.json:   0%|          | 0.00/715 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.44G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/227 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/14.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

### Post-processing on the model

We need to apply some post-processing on the 8-bit model to enable training, let's freeze all our layers, and cast the layer-norm in `float32` for stability. We also cast the output of the last layer in `float32` for the same reasons.

In [3]:
for param in model.parameters():
  param.requires_grad = False  # freeze the model - train adapters later
  if param.ndim == 1:
    # cast the small parameters (e.g. layernorm) to fp32 for stability
    param.data = param.data.to(torch.float32)

model.gradient_checkpointing_enable()  # reduce number of stored activations
model.enable_input_require_grads()

class CastOutputToFloat(nn.Sequential):
  def forward(self, x): return super().forward(x).to(torch.float32)
model.lm_head = CastOutputToFloat(model.lm_head)

### Apply LoRA

Here comes the magic with `peft`! Let's load a `PeftModel` and specify that we are going to use low-rank adapters (LoRA) using `get_peft_model` utility function from `peft`.

In [4]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [5]:
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["query_key_value"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)
print_trainable_parameters(model)

trainable params: 3145728 || all params: 1725554688 || trainable%: 0.18230242262828822


### Preprocessing

We can simply load our dataset from 🤗 Hugging Face with the `load_dataset` method!

In [6]:
import transformers
from datasets import load_dataset

dataset_name = "c-s-ale/Product-Descriptions-and-Ads"
product_name = "product"
product_desc = "description"
product_ad = "ad"

In [7]:
dataset = load_dataset(dataset_name)
print(dataset)
print(dataset['train'][0])

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/90 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/10 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['product', 'description', 'ad'],
        num_rows: 90
    })
    test: Dataset({
        features: ['product', 'description', 'ad'],
        num_rows: 10
    })
})
{'product': ' Harem pants', 'description': ' A style of pants with a dropped crotch, loose-fitting legs, and a gathered waistband for a unique, bohemian look.', 'ad': 'Discover Harem Pants! Unique, stylish bohemian vibes with a dropped crotch & loose legs. Comfy meets chic - elevate your wardrobe. Limited stock - shop now!'}


In [13]:
def generate_prompt(product_name: str, desc: str, ad: str) -> str:
  prompt = f"You are the world's best Marketing expert. Below is a product and description, please write an exceptional ad for this product which will capture the readers attention and increase the product recall value.\n\n### Product and Description:\n{product_name}: {desc}\n\n### Ad:\n{ad}"
  return prompt

dataset = dataset.map(lambda samples: tokenizer(generate_prompt(samples['product'], samples['description'], samples['ad'])))

Map:   0%|          | 0/90 [00:00<?, ? examples/s]

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

In [14]:
trainer = transformers.Trainer(
    model=model,
    train_dataset=dataset['train'],
    args=transformers.TrainingArguments(
        per_device_train_batch_size=4,
        gradient_accumulation_steps=4,
        warmup_steps=100,
        max_steps=100,
        learning_rate=1e-3,
        fp16=True,
        logging_steps=1,
        output_dir='outputs'
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False)
)
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
trainer.train()

Step,Training Loss
1,1.727600
2,1.715400
3,1.704200
4,1.619200
5,1.485500
6,1.382500
7,1.237000
8,1.132900
9,1.023500
10,0.911400


TrainOutput(global_step=100, training_loss=0.25125052250921726, metrics={'train_runtime': 397.8776, 'train_samples_per_second': 4.021, 'train_steps_per_second': 0.251, 'total_flos': 1402104216551424.0, 'train_loss': 0.25125052250921726, 'epoch': 17.39})

## Share adapters on the 🤗 Hub

In [15]:
HUGGING_FACE_USER_NAME = ""

In [16]:
from huggingface_hub import notebook_login
notebook_login()

In [17]:
model.push_to_hub(f"{HUGGING_FACE_USER_NAME}/AdPulse", use_auth_token=True)

adapter_model.safetensors:   0%|          | 0.00/12.6M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/abhamidi-1234/AdPulse/commit/e6f8298c4257529a2c3c38430a121ed154100307', commit_message='Upload model', commit_description='', oid='e6f8298c4257529a2c3c38430a121ed154100307', pr_url=None, pr_revision=None, pr_num=None)

## Load adapters from the Hub

You can also directly load adapters from the Hub using the commands below:

In [18]:
import torch
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer

peft_model_id = f"{HUGGING_FACE_USER_NAME}/AdPulse"
config = PeftConfig.from_pretrained(peft_model_id)
model = AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path, return_dict=True, load_in_8bit=True, device_map='auto')
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)

# Load the Lora model
model = PeftModel.from_pretrained(model, peft_model_id)

adapter_config.json:   0%|          | 0.00/568 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/222 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/14.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/12.6M [00:00<?, ?B/s]

## Inference

You can then directly use the trained model or the model that you have loaded from the 🤗 Hub for inference as you would do it usually in `transformers`.

### Take it for a spin!

In [19]:
from IPython.display import display, Markdown

def make_inference(product_name, product_description):
  batch = tokenizer(f"### Product and Description:\n{product_name}: {product_description}\n\n### Ad:", return_tensors='pt')

  with torch.cuda.amp.autocast():
    output_tokens = model.generate(**batch, max_new_tokens=50)

  display(Markdown((tokenizer.decode(output_tokens[0], skip_special_tokens=True))))

### Example in Training Set

In [21]:
batch = tokenizer("### Product and Description:\n Lace-up sandals: Shoes featuring laces or ties that wrap around the foot and, in some cases, the ankle.\n\n### Ad:", return_tensors='pt')

with torch.cuda.amp.autocast():
  output_tokens = model.generate(**batch, max_new_tokens=50)

print('\n\n', tokenizer.decode(output_tokens[0], skip_special_tokens=True))



 ### Product and Description:
 Lace-up sandals: Shoes featuring laces or ties that wrap around the foot and, in some cases, the ankle.

### Ad:
Introducing our latest Lace-up Sandals - the perfect blend of style and comfort! Wrap your feet in chic designs and experience unbeatable support. Perfect for summer! Shop now! Limited stock. Walk inelegance. Shop now


### Example outside of Training Set

In [22]:
batch = tokenizer("### Product and Description:\nSundress: A flowery yellow sundress with blue polka dots. \n\n### Ad:", return_tensors='pt')

with torch.cuda.amp.autocast():
  output_tokens = model.generate(**batch, max_new_tokens=50)

print('\n\n', tokenizer.decode(output_tokens[0], skip_special_tokens=True))



 ### Product and Description:
Sundress: A flowery yellow sundress with blue polka dots. 

### Ad:
Discover timeless style & unbeatable prices with our chic yellow sundresses & blue polka dots! Step into fashion-forward sophistication with these chicly flowery pieces. Shop now & make a statement in every way! �


### Example outside of immediate domain

In [23]:
batch = tokenizer("### Product and Description:\n A new Lexus: A luxury automobile with grey paint and tinted windows.\n\n### Ad:", return_tensors='pt')

with torch.cuda.amp.autocast():
  output_tokens = model.generate(**batch, max_new_tokens=50)

print('\n\n', tokenizer.decode(output_tokens[0], skip_special_tokens=True))



 ### Product and Description:
 A new Lexus: A luxury automobile with grey paint and tinted windows.

### Ad:
Introducing the Lexus NX300h! A luxurious, stylish car that combines sleek design with cutting-edge technology. Experience ultimate comfort and a lavishly upgraded interior with black paint and tinted windows. Get yours today


As you can see by fine-tuning for few steps we have almost recovered the exact quote from the training data.